

#CNN1

In [68]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.nn.functional as F



In [69]:
# Loading of the MNIST dataset
train_data = datasets.MNIST(
    root = 'data',  # root: The root directory where the dataset should be stored. In this case, it is set to 'data'. If the 'data' directory doesn't exist, the dataset will be downloaded to this location.
    train = True,
    transform = transforms.ToTensor(),  # transform: This parameter applies transformations to the data. In this case, transforms.ToTensor() is used to convert the images to PyTorch tensors.
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = transforms.ToTensor()
)

In [70]:
#Setting of the main hyper-parameters of the model
batch_size = 4 # The number of samples in each mini-batch used during training. Smaller batch sizes can lead to faster convergence but may introduce more noise into the training process.
n_train = batch_size * 125    # The total size of the training dataset. It's calculated as the product of batch_size and the number of batches (125 in this case). Adjusting the training dataset size can impact the model's ability to generalize.
n_test = batch_size * 25     # The total size of the test dataset. Similar to n_train, it's calculated as the product of batch_size and the number of test batches (25 in this case). The test dataset is used to evaluate the model's performance on unseen data.
n_channels = 4  # The number of channels in the output of the quantum convolution layer. In your model, you have set it to 4. This parameter determines the depth of the feature maps produced by the convolutional layer.
initial_lr =  0.005     # The initial learning rate for the stochastic gradient descent (SGD) optimizer.

In [71]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layer 1 with 1 input channels, 4 output channels, and 2x2 kernel
        self.conv = nn.Conv2d(1, 1, 4, stride=4)
        self.fc = nn.Linear(1 * 7 * 7, 10)

    def forward(self, x):
        # Propagate the input through the CNN layers
        x = self.conv(x)
        # Flatten the output from the convolutional layer
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc(x))
        return x


In [72]:
cnn = Net()
dataset  = train_data
train_size = n_train
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
for data in train_loader:
    inputs, labels = data
    print(f"{inputs.shape=}")
    print(f"{labels=}")
    outputs = cnn(inputs)
    print(f"{outputs.shape=}")
    print(f"{outputs=}")
    break

inputs.shape=torch.Size([4, 1, 28, 28])
labels=tensor([5, 8, 7, 0])
outputs.shape=torch.Size([4, 10])
outputs=tensor([[0.0000, 0.0967, 0.0000, 0.0376, 0.0762, 0.0462, 0.0965, 0.0399, 0.0115,
         0.0000],
        [0.0145, 0.0863, 0.0000, 0.0000, 0.0000, 0.0170, 0.1100, 0.0616, 0.0000,
         0.0000],
        [0.0330, 0.0799, 0.0000, 0.0884, 0.0000, 0.0714, 0.0843, 0.1359, 0.0000,
         0.0000],
        [0.0000, 0.1155, 0.0000, 0.0000, 0.0000, 0.0264, 0.1649, 0.0000, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)


In [73]:
# Train the model

import datetime
import os

dataset  = train_data

# Initialize your QCNN model
cnn = Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(cnn.parameters(), lr=initial_lr, momentum=0.90)  # Stochastic Gradient Descent optimizer
# Create a learning rate scheduler
# Here, we use StepLR which reduces the learning rate by a factor every step_size epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.0)
# Split your data into training and validation sets
train_size = n_train
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "ImgClass-Quanvolv.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

RESUME_TRAINING = True

# Training loop
num_epochs = 20
loss_list = []
cnn.train()

if RESUME_TRAINING is False:
    print(f"Restore model state from {MODEL_SAVE_PATH}")
    if os.path.exists(MODEL_SAVE_PATH):
        model_dict = torch.load(MODEL_SAVE_PATH)
        initial_epoch = model_dict['epoch'] + 1
        cnn.load_state_dict(model_dict['model_state_dict'])
        optimizer.load_state_dict(model_dict['optimizer_state_dict'])
        loss_list = model_dict['loss'].copy()
    else:
        print(f"No saved model state found. Training from scratch.")
        initial_epoch = 0
        loss_list = []
else:
    initial_epoch = 0
    loss_list = []

for epoch in range(num_epochs):
    ct = datetime.datetime.now()
    # Decay Learning Rate
    optimizer.step()
    scheduler.step()
    lr = scheduler.get_last_lr()
    print(f"{epoch=}, {lr=}, {ct}")
    running_loss = []
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # Zero the parameter gradients to avoid accumulation
        outputs = cnn(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        running_loss.append(loss.item())
        optimizer.step()  # Update the model parameters
    loss_list.append(sum(running_loss) / len(running_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(100. * (epoch + 1) / num_epochs, loss_list[-1]))
    torch.save({
        'epoch': epoch,
        'model_state_dict': cnn.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss_list,
    }, MODEL_SAVE_PATH)
    print(f"Saving model state to {MODEL_SAVE_PATH}")

print('Finished Training')

epoch=0, lr=[0.005], 2024-01-06 04:47:25.174867
Training [5%]	Loss: 2.2966
Saving model state to models/ImgClass-Quanvolv.pth
epoch=1, lr=[0.005], 2024-01-06 04:47:25.448558
Training [10%]	Loss: 2.2018
Saving model state to models/ImgClass-Quanvolv.pth
epoch=2, lr=[0.005], 2024-01-06 04:47:25.687253
Training [15%]	Loss: 1.5487
Saving model state to models/ImgClass-Quanvolv.pth
epoch=3, lr=[0.005], 2024-01-06 04:47:25.974222
Training [20%]	Loss: 1.0240
Saving model state to models/ImgClass-Quanvolv.pth
epoch=4, lr=[0.005], 2024-01-06 04:47:26.233153
Training [25%]	Loss: 0.8959
Saving model state to models/ImgClass-Quanvolv.pth
epoch=5, lr=[0.005], 2024-01-06 04:47:26.618003
Training [30%]	Loss: 0.7970
Saving model state to models/ImgClass-Quanvolv.pth
epoch=6, lr=[0.005], 2024-01-06 04:47:26.983339
Training [35%]	Loss: 0.7453
Saving model state to models/ImgClass-Quanvolv.pth
epoch=7, lr=[0.005], 2024-01-06 04:47:27.339387
Training [40%]	Loss: 0.7120
Saving model state to models/ImgClas

In [74]:
cnn.state_dict()

OrderedDict([('conv.weight',
              tensor([[[[ 1.4048,  0.3264,  1.0383,  1.2971],
                        [ 1.1972,  0.3413,  0.8934,  0.2260],
                        [ 0.3179,  0.7019,  0.4972, -0.0530],
                        [ 0.4917, -0.2844,  0.6161,  0.6083]]]])),
             ('conv.bias', tensor([-1.2372])),
             ('fc.weight',
              tensor([[ 3.9516e-02, -8.9391e-03,  2.9735e-03, -7.2266e-02, -1.0897e-01,
                        4.0562e-02,  1.3330e-02, -1.3625e-02,  2.5555e-01,  8.4740e-02,
                        2.7625e-01,  3.2129e-01, -3.1033e-02, -2.0671e-01,  7.9392e-02,
                       -5.4600e-01,  2.4809e-01,  4.2205e-01,  5.9236e-02,  2.6793e-01,
                        1.6021e-02,  1.3742e-01,  9.3207e-02,  6.1311e-01, -1.1547e+00,
                       -6.1099e-01,  6.1775e-01,  2.1080e-01, -2.0552e-02,  2.5187e-01,
                        7.2714e-01, -1.0613e+00,  3.2208e-02,  2.1923e-01,  3.7639e-01,
                       -7.70

In [81]:
#accuracy

# Use a small subset of the full validation dataset
from torch.utils.data import SubsetRandomSampler

K = n_test # enter your length here
subsample_train_indices = torch.randperm(len(val_set))[:K]
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, sampler=SubsetRandomSampler(subsample_train_indices))

correct = 0
total = 0
# Set the model to evaluation mode
cnn.eval()
with torch.inference_mode():
    for data in val_loader:
        images, labels = data
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy on the validation set: {100 * correct / total:.2f}%')

Accuracy on the validation set: 79.00%
